In [242]:
import zipfile
import requests,json
import io,StringIO
import sys,os
import re
import numpy

## FUNCTIONS

In [226]:
def get_file_content(file_url=None,directory=None,
                     fname='Par_file',
                     json_format=False,zip_format=False):
    if zip_format:
        try:
            try:
                response = requests.get(file_url)
                zfiles=zipfile.ZipFile(io.BytesIO(response.content))
            except:
                zfiles=zipfile.ZipFile(file_url)
            listfile = zfiles.namelist()
        except:
             print('ERROR: Error reading zip file %s' % file_url)
        
        try:
            parfound=False
            for f in listfile:
                if fname in f:
                    ifile = zfiles.read(f)
                    parfound=True
                    break
        except KeyError:
            parfound=False
        
        if not parfound: 
            print('ERROR: Did not find %s in zip file' % fname)
            return None

    elif directory:
        try:
            f.open(os.path.join(directory,fname),'r')
            ifile=f.read()
        except:
            print('ERROR: Error reading %s file in %s' % (fname,directory))
    elif json_format:
        try:
            try:
                response = requests.get(file_url)
                ifile=response.json()
            except:
                ifile=json.loads(file_url)
        except:
             print('ERROR: Error reading json file %s' % file_url)
    return ifile

In [138]:
def get_parameter(parameter=None,string=None,reg='\s+=([\s\d\.]+)'):
    if parameter is None: 
        print('Parameter missed, ex: parameter="NPROC"')
        return None
    elif string is None:
        print('file string missing')
    txt=parameter+reg
    p=re.findall(txt, string,re.MULTILINE)
    if len(p) > 0:
        return p[0]
    else:
        print('no parameter found')
        return None

In [163]:
def create_event_time(event_time):
    regex = r"PDE\s[\d\.\s]+"
    event_time=re.findall(regex, cmtsolution,re.MULTILINE)[0].split()[1:7]
    return '-'.join(x.zfill(2) for x in event_time[:3]) +\
            'T' +\
            ':'.join(x.zfill(2) for x in event_time[3:-1]) +':'+\
            '.'.join(x.zfill(2) for x in event_time[-1].split('.'))

In [198]:
def get_coordlimits(mesh):
    d = numpy.fromstring(mesh, sep=' ')
    data = d[1:].reshape(int(d[0]), 4)
    return data[:, 1].min(), data[:, 1].max(),\
        data[:, 2].min(), data[:, 2].max(),\
        data[:, 3].min(), data[:, 3].max()

In [ ]:
def get_mesh_geolimits(lim,epsg):
    import pyproj
    wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
    meshcoordinate=pyproj.Proj("+init="+epsg) 
    pyproj.transform(meshcoordinate, wgs84, lim[0], lim[2])
    longitude_min=max(pyproj.transform(meshcoordinate, wgs84, lim[0], lim[2])[0],
                  pyproj.transform(meshcoordinate, wgs84, lim[0], lim[3])[0])
    longitude_max=min(pyproj.transform(meshcoordinate, wgs84, lim[1], lim[2])[0],
                  pyproj.transform(meshcoordinate, wgs84, lim[1], lim[3])[0])
    latitude_min=max(pyproj.transform(meshcoordinate, wgs84, lim[0], lim[2])[1],
                  pyproj.transform(meshcoordinate, wgs84, lim[1], lim[2])[1])
    latitude_max=min(pyproj.transform(meshcoordinate, wgs84, lim[0], lim[3])[1],
                  pyproj.transform(meshcoordinate, wgs84, lim[1], lim[3])[1])
    return longitude_min,longitude_max,latitude_min,latitude_max

## GET CONTENT

In [84]:
data_url='https://gitlab.com/project-dare/dare-api/raw/master/containers/specfem3d/data.zip' 

In [228]:
cmtsolution=get_file_content(file_url=data_url,fname='CMTSOLUTION',zip_format=True)

In [165]:
parfile=get_file_content(file_url=data_url,zip_format=True)

In [222]:
infopar=get_file_content(file_url='https://gitlab.com/project-dare/WP6_EPOS/raw/master/specfem3d/specfem3d_input/DATA/mesh_Abruzzo/Info.json',
                directory=None,json_format=True)



In [229]:
meshfile=get_file_content(file_url=data_url,directory=None,fname='nodes_coords',zip_format=True)

## GET PARAMETERS

In [237]:
NPROC=int(get_parameter('NPROC',parfile))
dt=float(get_parameter('DT',parfile))
nstep=int(get_parameter('NSTEP',parfile))
try:
    RECORD_LENGTH_IN_MINUTES=float(get_parameter('RECORD_LENGTH_IN_MINUTES',parfile))
except:
    RECORD_LENGTH_IN_MINUTES=dt*nstep/60.
print(nproc,dt,nstep,RECORD_LENGTH_IN_MINUTES)

no parameter found
(24, 0.005, 12000, 1.0)


In [238]:
ETIME=create_event_time(event_time)
print ETIME

2013-02-16T21:16:09.29


In [208]:
lim=get_coordlimits(meshfile)

In [223]:
epsg=infopar['Coordinatesystem']['EPSG']
longitude_min,longitude_max,latitude_min,latitude_max=get_mesh_geolimits(lim,epsg)
xmin,xmax,ymin,ymax,zmin,zmax=lim

## CREATE JSON

In [239]:
d={
"simulationRunId": None,
"runId": None,
"nproc": NPROC,
"downloadPE": [
{
"input": {
"minimum_interstation_distance_in_m": 100,
"channel_priorities": [
"BH[E,N,Z]",
"EH[E,N,Z]"
],
"location_priorities": [
"",
"00",
"10"
],
"mseed_path": "./data",
"stationxml_path": "./stations",
"RECORD_LENGTH_IN_MINUTES": RECORD_LENGTH_IN_MINUTES,
"ORIGIN_TIME": ETIME,
"minlatitude": latitude_min,
"maxlatitude": latitude_max,
"minlongitude": longitude_min,
"maxlongitude": longitude_min
}
}
]
}


In [240]:
with open("data_file.json", "w") as write_file:
    json.dump(d, write_file)

In [241]:
!cat data_file.json

{"simulationRunId": null, "downloadPE": [{"input": {"channel_priorities": ["BH[E,N,Z]", "EH[E,N,Z]"], "minlongitude": 12.041644551237324, "minimum_interstation_distance_in_m": 100, "minlatitude": 41.10007459633125, "location_priorities": ["", "00", "10"], "stationxml_path": "./stations", "maxlongitude": 12.041644551237324, "RECORD_LENGTH_IN_MINUTES": 1.0, "maxlatitude": 42.897779709480716, "mseed_path": "./data", "ORIGIN_TIME": "2013-02-16T21:16:09.29"}}], "runId": null, "nproc": 24}